In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Aim is to predict the marks of students of the test data

In [20]:

data = pd.read_excel('Training data.xlsx')
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)
y_train = y_train.reshape(x_train.shape[0])

def feature_changing(x_train):
  # ---------
  study_ratio = x_train[:,3]/(x_train[:,2] + x_train[:,3] + x_train[:,4])
  study_ratio = study_ratio.reshape(-1,1)

  x_train = np.append(x_train, study_ratio, axis=1)
  for i in range(x_train.shape[0]):
    for j in range(2):
      if ((x_train[i,j] == 'no') or (x_train[i,j] == 'F')):
        x_train[i,j] = 0
      else:
        x_train[i,j] = 1
  # ---------
  return x_train

x_train = feature_changing(x_train)


In [21]:
def z_score(x_train):

    x_sum = np.sum(x_train,axis=0)
    x_mean = x_sum/x_train.shape[0]
    sum = np.zeros(9)
    for i in range(9):
      arr_temp = (x_train[:,i] - x_mean[i])**2
      sum[i] = np.sum(arr_temp, axis=0)
    sum = sum/x_train.shape[0]
    x_std = np.sqrt(sum)
    for i in range(9):
      x_train[:,i] = (x_train[:,i] - x_mean[i])/x_std[i]
    return x_train,x_std,x_mean


In [22]:
def cost(x_train,y_train,w,b):

    loss_arr = (np.matmul(x_train,w) + b - y_train)**2
    loss = np.sum(loss_arr)

    loss = loss/(2*x_train.shape[0])

    return loss

In [23]:
def gradient_descent(x_train,y_train,w,b):

  # ---------
    arr_temp = np.matmul(x_train,w) + b - y_train
    arr_temp = np.matmul(x_train.T, arr_temp)
    dJdw = arr_temp.T/x_train.shape[0]

    arr = np.matmul(x_train,w) + b - y_train
    dJdb = np.sum(arr)/x_train.shape[0]
    alpha = 0.01
    w = w - (alpha*dJdw)
    b = b - (alpha*dJdb)

    return w,b

In [24]:
x_train = x_train.astype(np.float64)
x_train,x_std,x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1],1)
w = w.reshape(9)
b = np.random.randn(1)

old_cost = 0

while abs(old_cost - cost(x_train,y_train,w,b))>0.00001:
  old_cost = cost(x_train,y_train,w,b)
  w,b = gradient_descent(x_train,y_train,w,b)

x_predict = pd.read_excel('Test data.xlsx').iloc[:,:8].to_numpy()
x_predict = feature_changing(x_predict)
for i in range(9):
  x_predict[:,i] = (x_predict[:,i] - x_mean[i])/x_std[i]
#x_predict = (x_predict - x_mean)/x_std
ans = pd.read_excel('Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict,w) + b

accuracy = 0
for dim in range(len(ans)):
  if abs(y_predict[dim]-ans[dim])<0.5:
    accuracy += 1
accuracy = round(accuracy*100/200.0,2)
ok = 'Congratulations' if accuracy>95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

Congratulations, your accuracy is 97.0%
